# Trendyol Hackathon 2025 Kaggle Phase Team SKY Solution

- This notebook contains solution that merges two different approaches.
- First approach is made by [Kaan Durmuşoğlu (kulagimcinliyo)](https://www.kaggle.com/kulagimcinliyo)
- Second approach is made by [Bilalcan Ustabaş (bilalcanustaba)](https://www.kaggle.com/bilalcanustaba)
- At the end, both approaches are ensembled.

## Libraries

In [ ]:
import polars as pl
import pandas as pd
import numpy as np
import duckdb

from helpers.decay_features import add_decay_features_multiple
from helpers.user_history import add_user_history, add_user_term_history, add_user_metadata
from helpers.content_history import add_content_price_history
from helpers.session_history import candidate_counter, session_based_ranking_for_contents
from helpers.time_history import add_time_history

from catboost import CatBoostRanker

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

DATA_PATH = "../data"

## 1st Solution (Kaan)

### Functions

In [ ]:
# --- DuckDB SQL Query Generation ---
def get_base_query(table_name: str) -> str:
    """Generates the base query for train or test data."""
    return f"""
    base AS (
        SELECT
            *,
            ts_hour AS date
        FROM read_parquet('{DATA_PATH}/{table_name}')
    )
    """

def get_feature_queries() -> str:
    """
    Generates the SQL for all feature engineering CTEs.
    """
    return f"""
    -- 1. USER SITEWIDE LOG FEATURES
    user_sitewide_log_rolled AS (
        SELECT
            user_id_hashed, ts_hour,
            SUM(total_click) OVER w AS rolling_total_click_24h,
            SUM(total_order) OVER w AS rolling_total_order_24h,
            SUM(total_cart) OVER w AS rolling_total_cart_24h,
            SUM(total_fav) OVER w AS rolling_total_fav_24h,
            SUM(total_click) OVER w2 AS rolling_total_click_72h,
            SUM(total_order) OVER w2 AS rolling_total_order_72h,
            SUM(total_cart) OVER w2 AS rolling_total_cart_72h,
            SUM(total_fav) OVER w2 AS rolling_total_fav_72h
        FROM read_parquet('{DATA_PATH}/user/sitewide_log.parquet')
        WINDOW
            w AS (PARTITION BY user_id_hashed ORDER BY ts_hour RANGE BETWEEN INTERVAL '24' HOUR PRECEDING AND INTERVAL '1' MICROSECOND PRECEDING),
            w2 AS (PARTITION BY user_id_hashed ORDER BY ts_hour RANGE BETWEEN INTERVAL '72' HOUR PRECEDING AND INTERVAL '1' MICROSECOND PRECEDING)
    ),
    user_sitewide_features AS (
        SELECT
            user_id_hashed, ts_hour,
            COALESCE(rolling_total_order_24h / NULLIF(rolling_total_click_24h, 0), 0) AS click_to_order_24h_user_sitewide_log,
            COALESCE(rolling_total_cart_24h / NULLIF(rolling_total_click_24h, 0), 0) AS click_to_cart_24h_user_sitewide_log,
            COALESCE(rolling_total_order_24h / NULLIF(rolling_total_cart_24h, 0), 0) AS cart_to_order_24h_user_sitewide_log,
            COALESCE(rolling_total_order_72h / NULLIF(rolling_total_click_72h, 0), 0) AS click_to_order_72h_user_sitewide_log,
            COALESCE(rolling_total_cart_72h / NULLIF(rolling_total_click_72h, 0), 0) AS click_to_cart_72h_user_sitewide_log,
            COALESCE(rolling_total_order_72h / NULLIF(rolling_total_cart_72h, 0), 0) AS cart_to_order_72h_user_sitewide_log
        FROM user_sitewide_log_rolled
    ),

    -- 2. USER SEARCH LOG FEATURES
    user_search_log_rolled AS (
        SELECT
            user_id_hashed, ts_hour,
            SUM(total_search_impression) OVER w AS rolling_total_search_impression_24h,
            SUM(total_search_click) OVER w AS rolling_total_search_click_24h,
            SUM(total_search_impression) OVER w2 AS rolling_total_search_impression_72h,
            SUM(total_search_click) OVER w2 AS rolling_total_search_click_72h
        FROM read_parquet('{DATA_PATH}/user/search_log.parquet')
        WINDOW
            w AS (PARTITION BY user_id_hashed ORDER BY ts_hour RANGE BETWEEN INTERVAL '24' HOUR PRECEDING AND INTERVAL '1' MICROSECOND PRECEDING),
            w2 AS (PARTITION BY user_id_hashed ORDER BY ts_hour RANGE BETWEEN INTERVAL '72' HOUR PRECEDING AND INTERVAL '1' MICROSECOND PRECEDING)
    ),
    user_search_features AS (
        SELECT
            user_id_hashed, ts_hour,
            COALESCE(rolling_total_search_click_24h / NULLIF(rolling_total_search_impression_24h, 0), 0) AS search_ctr_24h_user_search_log,
            COALESCE(rolling_total_search_click_72h / NULLIF(rolling_total_search_impression_72h, 0), 0) AS search_ctr_72h_user_search_log
        FROM user_search_log_rolled
    ),

    -- 3. USER METADATA (Static, pre-processed to handle invalid birth years)
    user_metadata AS (
        SELECT * FROM user_meta_df
    ),

    -- 4. USER TOP TERMS LOG
    user_top_terms_rolled AS (
        SELECT
            user_id_hashed, search_term_normalized, ts_hour,
            SUM(total_search_impression) OVER w AS rolling_total_search_impression_24h,
            SUM(total_search_click) OVER w AS rolling_total_search_click_24h,
            SUM(total_search_impression) OVER w2 AS rolling_total_search_impression_72h,
            SUM(total_search_click) OVER w2 AS rolling_total_search_click_72h
        FROM read_parquet('{DATA_PATH}/user/top_terms_log.parquet')
        WINDOW
            w AS (PARTITION BY user_id_hashed, search_term_normalized ORDER BY ts_hour RANGE BETWEEN INTERVAL '24' HOUR PRECEDING AND INTERVAL '1' MICROSECOND PRECEDING),
            w2 AS (PARTITION BY user_id_hashed, search_term_normalized ORDER BY ts_hour RANGE BETWEEN INTERVAL '72' HOUR PRECEDING AND INTERVAL '1' MICROSECOND PRECEDING)
    ),
    user_top_terms_features AS (
        SELECT
            user_id_hashed, search_term_normalized, ts_hour,
            COALESCE(rolling_total_search_click_24h / NULLIF(rolling_total_search_impression_24h, 0), 0) AS search_ctr_24h_user_top_terms_log,
            COALESCE(rolling_total_search_click_72h / NULLIF(rolling_total_search_impression_72h, 0), 0) AS search_ctr_72h_user_top_terms_log
        FROM user_top_terms_rolled
    ),

    -- 5. USER FASHION SITEWIDE LOG
    user_fashion_sitewide_log_rolled AS (
        SELECT
            user_id_hashed, content_id_hashed, ts_hour,
            SUM(total_click) OVER w AS rolling_total_click_24h,
            SUM(total_order) OVER w AS rolling_total_order_24h,
            SUM(total_cart) OVER w AS rolling_total_cart_24h,
            SUM(total_fav) OVER w AS rolling_total_fav_24h,
            SUM(total_click) OVER w2 AS rolling_total_click_72h,
            SUM(total_order) OVER w2 AS rolling_total_order_72h,
            SUM(total_cart) OVER w2 AS rolling_total_cart_72h,
            SUM(total_fav) OVER w2 AS rolling_total_fav_72h
        FROM read_parquet('{DATA_PATH}/user/fashion_sitewide_log.parquet')
        WINDOW
            w AS (PARTITION BY user_id_hashed, content_id_hashed ORDER BY ts_hour RANGE BETWEEN INTERVAL '24' HOUR PRECEDING AND INTERVAL '1' MICROSECOND PRECEDING),
            w2 AS (PARTITION BY user_id_hashed, content_id_hashed ORDER BY ts_hour RANGE BETWEEN INTERVAL '72' HOUR PRECEDING AND INTERVAL '1' MICROSECOND PRECEDING)
    ),
    user_fashion_sitewide_features AS (
        SELECT
            user_id_hashed, content_id_hashed, ts_hour,
            COALESCE(rolling_total_order_24h / NULLIF(rolling_total_click_24h, 0), 0) AS click_to_order_24h_user_fashion_sitewide_log,
            COALESCE(rolling_total_cart_24h / NULLIF(rolling_total_click_24h, 0), 0) AS click_to_cart_24h_user_fashion_sitewide_log,
            COALESCE(rolling_total_order_24h / NULLIF(rolling_total_cart_24h, 0), 0) AS cart_to_order_24h_user_fashion_sitewide_log,
            COALESCE(rolling_total_order_72h / NULLIF(rolling_total_click_72h, 0), 0) AS click_to_order_72h_user_fashion_sitewide_log,
            COALESCE(rolling_total_cart_72h / NULLIF(rolling_total_click_72h, 0), 0) AS click_to_cart_72h_user_fashion_sitewide_log,
            COALESCE(rolling_total_order_72h / NULLIF(rolling_total_cart_72h, 0), 0) AS cart_to_order_72h_user_fashion_sitewide_log
        FROM user_fashion_sitewide_log_rolled
    ),

    -- 6. USER FASHION SEARCH LOG
    user_fashion_search_log_rolled AS (
        SELECT
            user_id_hashed, content_id_hashed, ts_hour,
            SUM(total_search_click) OVER w AS rolling_total_search_click_24h,
            SUM(total_search_click) OVER w2 AS rolling_total_search_click_72h
        FROM (
            SELECT * FROM read_parquet('{DATA_PATH}/user/fashion_search_log.parquet')
            WHERE total_search_click > 0
            AND content_id_hashed IN (
                (SELECT DISTINCT content_id_hashed FROM read_parquet('{DATA_PATH}/train_sessions.parquet'))
                UNION
                (SELECT DISTINCT content_id_hashed FROM read_parquet('{DATA_PATH}/test_sessions.parquet'))
            )
        ) AS filtered_search
        WINDOW
            w AS (PARTITION BY user_id_hashed, content_id_hashed ORDER BY ts_hour RANGE BETWEEN INTERVAL '24' HOUR PRECEDING AND INTERVAL '1' MICROSECOND PRECEDING),
            w2 AS (PARTITION BY user_id_hashed, content_id_hashed ORDER BY ts_hour RANGE BETWEEN INTERVAL '72' HOUR PRECEDING AND INTERVAL '1' MICROSECOND PRECEDING)
    ),

    -- 7. TERM SEARCH LOG
    term_search_log_rolled AS (
        SELECT
            search_term_normalized, ts_hour,
            SUM(total_search_impression) OVER w AS rolling_total_search_impression_24h,
            SUM(total_search_click) OVER w AS rolling_total_search_click_24h,
            SUM(total_search_impression) OVER w2 AS rolling_total_search_impression_72h,
            SUM(total_search_click) OVER w2 AS rolling_total_search_click_72h
        FROM read_parquet('{DATA_PATH}/term/search_log.parquet')
        WINDOW
            w AS (PARTITION BY search_term_normalized ORDER BY ts_hour RANGE BETWEEN INTERVAL '24' HOUR PRECEDING AND INTERVAL '1' MICROSECOND PRECEDING),
            w2 AS (PARTITION BY search_term_normalized ORDER BY ts_hour RANGE BETWEEN INTERVAL '72' HOUR PRECEDING AND INTERVAL '1' MICROSECOND PRECEDING)
    ),
    term_search_features AS (
        SELECT
            search_term_normalized, ts_hour,
            COALESCE(rolling_total_search_click_24h / NULLIF(rolling_total_search_impression_24h, 0), 0) AS search_ctr_24h_term_search_log,
            COALESCE(rolling_total_search_click_72h / NULLIF(rolling_total_search_impression_72h, 0), 0) AS search_ctr_72h_term_search_log
        FROM term_search_log_rolled
    ),

    -- 8. CONTENT SEARCH LOG
    content_search_log_rolled AS (
        SELECT
            content_id_hashed, date,
            SUM(total_search_impression) OVER w AS rolling_total_search_impression_3d,
            SUM(total_search_click) OVER w AS rolling_total_search_click_3d,
            SUM(total_search_impression) OVER w2 AS rolling_total_search_impression_7d,
            SUM(total_search_click) OVER w2 AS rolling_total_search_click_7d
        FROM read_parquet('{DATA_PATH}/content/search_log.parquet')
        WINDOW
            w AS (PARTITION BY content_id_hashed ORDER BY date RANGE BETWEEN INTERVAL '3' DAY PRECEDING AND INTERVAL '1' MICROSECOND PRECEDING),
            w2 AS (PARTITION BY content_id_hashed ORDER BY date RANGE BETWEEN INTERVAL '7' DAY PRECEDING AND INTERVAL '1' MICROSECOND PRECEDING)
    ),
    content_search_features AS (
        SELECT
            content_id_hashed, date,
            COALESCE(rolling_total_search_click_3d / NULLIF(rolling_total_search_impression_3d, 0), 0) AS search_ctr_3d_content_search_log,
            COALESCE(rolling_total_search_click_7d / NULLIF(rolling_total_search_impression_7d, 0), 0) AS search_ctr_7d_content_search_log
        FROM content_search_log_rolled
    ),

    -- 9. CONTENT TOP TERMS LOG
    content_top_terms_rolled AS (
        SELECT
            content_id_hashed, search_term_normalized, date,
            SUM(total_search_impression) OVER w AS rolling_total_search_impression_24h,
            SUM(total_search_click) OVER w AS rolling_total_search_click_24h,
            SUM(total_search_impression) OVER w2 AS rolling_total_search_impression_72h,
            SUM(total_search_click) OVER w2 AS rolling_total_search_click_72h
        FROM read_parquet('{DATA_PATH}/content/top_terms_log.parquet')
        WINDOW
            w AS (PARTITION BY content_id_hashed, search_term_normalized ORDER BY date RANGE BETWEEN INTERVAL '24' HOUR PRECEDING AND INTERVAL '1' MICROSECOND PRECEDING),
            w2 AS (PARTITION BY content_id_hashed, search_term_normalized ORDER BY date RANGE BETWEEN INTERVAL '72' HOUR PRECEDING AND INTERVAL '1' MICROSECOND PRECEDING)
    ),
    content_top_terms_features AS (
        SELECT
            content_id_hashed, search_term_normalized, date,
            COALESCE(rolling_total_search_click_24h / NULLIF(rolling_total_search_impression_24h, 0), 0) AS search_ctr_24h_content_top_terms_log,
            COALESCE(rolling_total_search_click_72h / NULLIF(rolling_total_search_impression_72h, 0), 0) AS search_ctr_72h_content_top_terms_log
        FROM content_top_terms_rolled
    ),

    -- 10. CONTENT SITEWIDE LOG
    content_sitewide_log_rolled AS (
        SELECT
            content_id_hashed, date,
            SUM(total_click) OVER w AS rolling_total_click_3d,
            SUM(total_order) OVER w AS rolling_total_order_3d,
            SUM(total_cart) OVER w AS rolling_total_cart_3d,
            SUM(total_fav) OVER w AS rolling_total_fav_3d,
            SUM(total_click) OVER w2 AS rolling_total_click_7d,
            SUM(total_order) OVER w2 AS rolling_total_order_7d,
            SUM(total_cart) OVER w2 AS rolling_total_cart_7d,
            SUM(total_fav) OVER w2 AS rolling_total_fav_7d
        FROM read_parquet('{DATA_PATH}/content/sitewide_log.parquet')
        WINDOW
            w AS (PARTITION BY content_id_hashed ORDER BY date RANGE BETWEEN INTERVAL '3' DAY PRECEDING AND INTERVAL '1' MICROSECOND PRECEDING),
            w2 AS (PARTITION BY content_id_hashed ORDER BY date RANGE BETWEEN INTERVAL '7' DAY PRECEDING AND INTERVAL '1' MICROSECOND PRECEDING)
    ),
    content_sitewide_features AS (
        SELECT
            content_id_hashed, date,
            COALESCE(rolling_total_order_3d / NULLIF(rolling_total_click_3d, 0), 0) AS click_to_order_3d_content_sitewide_log,
            COALESCE(rolling_total_cart_3d / NULLIF(rolling_total_click_3d, 0), 0) AS click_to_cart_3d_content_sitewide_log,
            COALESCE(rolling_total_order_3d / NULLIF(rolling_total_cart_3d, 0), 0) AS cart_to_order_3d_content_sitewide_log,
            COALESCE(rolling_total_order_7d / NULLIF(rolling_total_click_7d, 0), 0) AS click_to_order_7d_content_sitewide_log,
            COALESCE(rolling_total_cart_7d / NULLIF(rolling_total_click_7d, 0), 0) AS click_to_cart_7d_content_sitewide_log,
            COALESCE(rolling_total_order_7d / NULLIF(rolling_total_cart_7d, 0), 0) AS cart_to_order_7d_content_sitewide_log
        FROM content_sitewide_log_rolled
    ),

    -- 11. CONTENT METADATA (Static)
    content_metadata AS (
        SELECT * FROM read_parquet('{DATA_PATH}/content/metadata.parquet')
    ),

    -- 12. CONTENT PRICE DATA (Time-series, needs ASOF join)
    content_price AS (
        SELECT * FROM read_parquet('{DATA_PATH}/content/price_rate_review_data.parquet')
    ),

    -- 13. USER-CONTENT INTERACTION HISTORY
    user_content_action_stats AS (
        SELECT
            user_id_hashed, content_id_hashed, ts_hour,
            SUM(total_click) OVER w AS total_click_so_far,
            SUM(total_cart) OVER w AS total_cart_so_far,
            SUM(total_fav) OVER w AS total_fav_so_far,
            SUM(total_order) OVER w AS total_order_so_far,
            MAX(CASE WHEN total_click > 0 THEN ts_hour ELSE NULL END) OVER w AS last_click_ts_hour,
            MAX(CASE WHEN total_cart > 0 THEN ts_hour ELSE NULL END) OVER w AS last_cart_ts_hour,
            MAX(CASE WHEN total_fav > 0 THEN ts_hour ELSE NULL END) OVER w AS last_fav_ts_hour,
            MAX(CASE WHEN total_order > 0 THEN ts_hour ELSE NULL END) OVER w AS last_order_ts_hour
        FROM read_parquet('{DATA_PATH}/user/fashion_sitewide_log.parquet')
        WINDOW
            w AS (PARTITION BY user_id_hashed, content_id_hashed ORDER BY ts_hour RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
    ),

    -- 14. PREPARE HISTORICAL INTERACTIONS FOR USER-CATEGORY FEATURES
    historical_interactions AS (
        SELECT
            t.user_id_hashed,
            t.ts_hour,
            COALESCE(c.level1_category_name, 'UNKNOWN') AS level1_category_name,
            COALESCE(c.level2_category_name, 'UNKNOWN') AS level2_category_name,
            COALESCE(c.leaf_category_name, 'UNKNOWN') AS leaf_category_name,
            t.clicked,
            t.ordered
        FROM read_parquet('{DATA_PATH}/train_sessions.parquet') AS t
        LEFT JOIN content_metadata AS c ON t.content_id_hashed = c.content_id_hashed
    ),
    user_category_features_rolled AS (
        SELECT
            user_id_hashed,
            ts_hour,
            level1_category_name,
            level2_category_name,
            leaf_category_name,
            SUM(clicked) OVER (PARTITION BY user_id_hashed, level1_category_name ORDER BY ts_hour RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS user_l1_cat_clicks,
            SUM(ordered) OVER (PARTITION BY user_id_hashed, level1_category_name ORDER BY ts_hour RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS user_l1_cat_orders,
            SUM(clicked) OVER (PARTITION BY user_id_hashed, level2_category_name ORDER BY ts_hour RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS user_l2_cat_clicks,
            SUM(ordered) OVER (PARTITION BY user_id_hashed, level2_category_name ORDER BY ts_hour RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS user_l2_cat_orders,
            SUM(clicked) OVER (PARTITION BY user_id_hashed, leaf_category_name ORDER BY ts_hour RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS user_leaf_cat_clicks,
            SUM(ordered) OVER (PARTITION BY user_id_hashed, leaf_category_name ORDER BY ts_hour RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS user_leaf_cat_orders
        FROM historical_interactions
    ),
    user_category_features AS (
        SELECT
            user_id_hashed, ts_hour, level1_category_name, level2_category_name, leaf_category_name,
            COALESCE(user_l1_cat_orders / NULLIF(user_l1_cat_clicks, 0), 0) AS user_l1_cat_conversion_rate,
            COALESCE(user_l2_cat_orders / NULLIF(user_l2_cat_clicks, 0), 0) AS user_l2_cat_conversion_rate,
            COALESCE(user_leaf_cat_orders / NULLIF(user_leaf_cat_clicks, 0), 0) AS user_leaf_cat_conversion_rate,
            user_l1_cat_clicks, user_l1_cat_orders
        FROM user_category_features_rolled
    ),
    
    -- 15. CONTENT POPULARITY VELOCITY & ALL-TIME STATS
    content_popularity_rolled AS (
        SELECT
            content_id_hashed,
            date,
            SUM(total_order) OVER w7 AS rolling_total_order_7d,
            SUM(total_click) OVER w7 AS rolling_total_click_7d,
            SUM(total_order) OVER w30 AS rolling_total_order_30d,
            SUM(total_click) OVER w30 AS rolling_total_click_30d,
            SUM(total_order) OVER w_all AS all_time_total_order,
            SUM(total_click) OVER w_all AS all_time_total_click,
            SUM(total_cart) OVER w_all AS all_time_total_cart,
            SUM(total_fav) OVER w_all AS all_time_total_fav
        FROM read_parquet('{DATA_PATH}/content/sitewide_log.parquet')
        WINDOW
            w7 AS (PARTITION BY content_id_hashed ORDER BY date RANGE BETWEEN INTERVAL '7' DAY PRECEDING AND INTERVAL '1' MICROSECOND PRECEDING),
            w30 AS (PARTITION BY content_id_hashed ORDER BY date RANGE BETWEEN INTERVAL '30' DAY PRECEDING AND INTERVAL '1' MICROSECOND PRECEDING),
            w_all AS (PARTITION BY content_id_hashed ORDER BY date RANGE BETWEEN UNBOUNDED PRECEDING AND INTERVAL '1' MICROSECOND PRECEDING)
    ),
    content_popularity_features AS (
        SELECT
            content_id_hashed,
            date,
            all_time_total_order,
            all_time_total_click,
            all_time_total_cart,
            all_time_total_fav,
            COALESCE((rolling_total_order_7d / 7.0) / NULLIF(rolling_total_order_30d / 30.0, 0), 1) AS order_velocity_7d_vs_30d,
            COALESCE((rolling_total_click_7d / 7.0) / NULLIF(rolling_total_click_30d / 30.0, 0), 1) AS click_velocity_7d_vs_30d,
            COALESCE(all_time_total_order / NULLIF(all_time_total_click, 0), 0) AS all_time_click_to_order_rate
        FROM content_popularity_rolled
    ),

    -- 16. CATEGORY-RELATIVE POPULARITY RANK
    content_category_interactions AS (
        SELECT
            log.content_id_hashed,
            log.date,
            meta.level1_category_name,
            meta.level2_category_name,
            meta.leaf_category_name,
            SUM(log.total_order) OVER (PARTITION BY log.content_id_hashed ORDER BY log.date RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS content_all_time_orders
        FROM read_parquet('{DATA_PATH}/content/sitewide_log.parquet') AS log
        INNER JOIN content_metadata AS meta ON log.content_id_hashed = meta.content_id_hashed
    ),
    content_category_rank_features AS (
        SELECT
            content_id_hashed,
            date,
            RANK() OVER (PARTITION BY leaf_category_name, date ORDER BY content_all_time_orders DESC) AS order_rank_in_leaf_category,
            RANK() OVER (PARTITION BY level2_category_name, date ORDER BY content_all_time_orders DESC) AS order_rank_in_l2_category,
            RANK() OVER (PARTITION BY level1_category_name, date ORDER BY content_all_time_orders DESC) AS order_rank_in_l1_category
        FROM content_category_interactions
    ),

    -- 17. COMBINE ALL USER-CONTENT INTERACTIONS FOR AGE ANALYSIS
    all_user_content_interactions AS (
        SELECT ts_hour, user_id_hashed, content_id_hashed, clicked, ordered, FALSE as search_clicked
        FROM read_parquet('{DATA_PATH}/train_sessions.parquet')
        WHERE clicked OR ordered
        UNION ALL
        SELECT ts_hour, user_id_hashed, content_id_hashed, (total_click > 0) as clicked, (total_order > 0) as ordered, FALSE as search_clicked
        FROM read_parquet('{DATA_PATH}/user/fashion_sitewide_log.parquet')
        WHERE total_click > 0 OR total_order > 0
        UNION ALL
        SELECT ts_hour, user_id_hashed, content_id_hashed, FALSE as clicked, FALSE as ordered, (total_search_click > 0) as search_clicked
        FROM read_parquet('{DATA_PATH}/user/fashion_search_log.parquet')
        WHERE total_search_click > 0
    ),

    -- 18. CALCULATE CONTENT-AGE FEATURES
    content_age_features_rolled AS (
        SELECT
            i.content_id_hashed,
            i.ts_hour,
            (EXTRACT(year FROM i.ts_hour) - u.user_birth_year) AS user_age_at_interaction
        FROM all_user_content_interactions AS i
        INNER JOIN user_metadata AS u ON i.user_id_hashed = u.user_id_hashed
        WHERE u.user_birth_year IS NOT NULL AND (i.clicked OR i.ordered OR i.search_clicked)
    ),
    content_age_features AS (
        SELECT
            content_id_hashed,
            ts_hour,
            AVG(user_age_at_interaction) OVER w AS avg_interaction_age_historical,
            STDDEV_SAMP(user_age_at_interaction) OVER w AS stddev_interaction_age_historical
        FROM content_age_features_rolled
        WINDOW
            w AS (PARTITION BY content_id_hashed ORDER BY ts_hour RANGE BETWEEN UNBOUNDED PRECEDING AND INTERVAL '1' MICROSECOND PRECEDING)
    ),

    -- 19. CONTENT-GENDER FEATURES
    content_gender_features_rolled AS (
        SELECT
            i.content_id_hashed,
            i.ts_hour,
            u.user_gender,
            SUM(CASE WHEN i.clicked OR i.search_clicked THEN 1 ELSE 0 END) OVER w AS gender_clicks,
            SUM(CASE WHEN i.ordered THEN 1 ELSE 0 END) OVER w AS gender_orders
        FROM all_user_content_interactions AS i
        INNER JOIN user_metadata AS u ON i.user_id_hashed = u.user_id_hashed
        WHERE u.user_gender IS NOT NULL AND (i.clicked OR i.ordered OR i.search_clicked)
        WINDOW
            w AS (PARTITION BY i.content_id_hashed, u.user_gender ORDER BY i.ts_hour RANGE BETWEEN UNBOUNDED PRECEDING AND INTERVAL '1' MICROSECOND PRECEDING)
    ),
    content_gender_features AS (
        SELECT
            content_id_hashed,
            ts_hour,
            MAX(CASE WHEN user_gender = 'Bay' THEN gender_clicks ELSE 0 END) AS male_clicks_historical,
            MAX(CASE WHEN user_gender = 'Bay' THEN gender_orders ELSE 0 END) AS male_orders_historical,
            MAX(CASE WHEN user_gender = 'Bayan' THEN gender_clicks ELSE 0 END) AS female_clicks_historical,
            MAX(CASE WHEN user_gender = 'Bayan' THEN gender_orders ELSE 0 END) AS female_orders_historical
        FROM content_gender_features_rolled
        GROUP BY 1, 2
    ),

    -- 20. CATEGORY-PRICE FEATURES
    category_price_features AS (
        SELECT
            cm.leaf_category_name,
            cp.update_date,
            AVG(cp.selling_price) OVER (PARTITION BY cm.leaf_category_name ORDER BY cp.update_date) as avg_price_in_leaf_category
        FROM content_price AS cp
        JOIN content_metadata AS cm ON cp.content_id_hashed = cm.content_id_hashed
    )
    """

def get_final_select_query() -> str:
    """
    Generates the final SELECT statement that joins all the feature CTEs.
    """
    return """
    SELECT
        base.*,
        -- --- SEARCH TERM FEATURES ---
        LENGTH(base.search_term_normalized) AS search_term_length,
        length(split(base.search_term_normalized, '_')) AS search_term_word_count,

        -- --- USER AGE FEATURE ---
        (EXTRACT(year FROM base.ts_hour) - um.user_birth_year) AS user_age,

        -- --- IN-SESSION FEATURES ---
        COUNT(base.content_id_hashed) OVER (PARTITION BY base.session_id) as session_item_count,
        (CASE WHEN cp.selling_price = MIN(cp.selling_price) OVER (PARTITION BY base.session_id) THEN 1 ELSE 0 END) AS is_cheapest_in_session,
        (CASE WHEN cp.selling_price = MAX(cp.selling_price) OVER (PARTITION BY base.session_id) THEN 1 ELSE 0 END) AS is_most_expensive_in_session,

        -- --- IN-SESSION STATISTICAL FEATURES ---
        AVG(cp.selling_price) OVER (PARTITION BY base.session_id) as session_avg_price,
        STDDEV_SAMP(cp.selling_price) OVER (PARTITION BY base.session_id) as session_std_price,
        MIN(cp.selling_price) OVER (PARTITION BY base.session_id) as session_min_price,
        MAX(cp.selling_price) OVER (PARTITION BY base.session_id) as session_max_price,

        AVG(cp.content_review_count) OVER (PARTITION BY base.session_id) as session_avg_review_count,
        STDDEV_SAMP(cp.content_review_count) OVER (PARTITION BY base.session_id) as session_std_review_count,
        MIN(cp.content_review_count) OVER (PARTITION BY base.session_id) as session_min_review_count,
        MAX(cp.content_review_count) OVER (PARTITION BY base.session_id) as session_max_review_count,

        AVG(cp.content_review_wth_media_count) OVER (PARTITION BY base.session_id) as session_avg_review_wth_media_count,
        STDDEV_SAMP(cp.content_review_wth_media_count) OVER (PARTITION BY base.session_id) as session_std_review_wth_media_count,
        MIN(cp.content_review_wth_media_count) OVER (PARTITION BY base.session_id) as session_min_review_wth_media_count,
        MAX(cp.content_review_wth_media_count) OVER (PARTITION BY base.session_id) as session_max_review_wth_media_count,

        AVG(cp.content_rate_avg) OVER (PARTITION BY base.session_id) as session_avg_rate,
        STDDEV_SAMP(cp.content_rate_avg) OVER (PARTITION BY base.session_id) as session_std_rate,
        MIN(cp.content_rate_avg) OVER (PARTITION BY base.session_id) as session_min_rate,
        MAX(cp.content_rate_avg) OVER (PARTITION BY base.session_id) as session_max_rate,

        AVG(cp.content_rate_count) OVER (PARTITION BY base.session_id) as session_avg_rate_count,
        STDDEV_SAMP(cp.content_rate_count) OVER (PARTITION BY base.session_id) as session_std_rate_count,
        MIN(cp.content_rate_count) OVER (PARTITION BY base.session_id) as session_min_rate_count,
        MAX(cp.content_rate_count) OVER (PARTITION BY base.session_id) as session_max_rate_count,

        -- in-session price features
        RANK() OVER (PARTITION BY base.session_id ORDER BY cp.selling_price ASC) AS price_rank,
        100.0 * (cp.selling_price - AVG(cp.selling_price) OVER (PARTITION BY base.session_id)) / NULLIF(AVG(cp.selling_price) OVER (PARTITION BY base.session_id), 0) AS price_diff_percent,

        -- in-session review features
        RANK() OVER (PARTITION BY base.session_id ORDER BY cp.content_rate_count ASC) AS content_rate_count_rank,
        RANK() OVER (PARTITION BY base.session_id ORDER BY cp.content_rate_avg ASC) AS content_rate_avg_rank,
        RANK() OVER (PARTITION BY base.session_id ORDER BY cp.content_review_count ASC) AS content_review_count_rank,
        RANK() OVER (PARTITION BY base.session_id ORDER BY cp.content_review_wth_media_count ASC) AS content_review_wth_media_count_rank,
        100.0 * (cp.content_rate_count - AVG(cp.content_rate_count) OVER (PARTITION BY base.session_id)) / NULLIF(AVG(cp.content_rate_count) OVER (PARTITION BY base.session_id), 0) AS content_rate_count_diff_percent,
        100.0 * (cp.content_rate_avg - AVG(cp.content_rate_avg) OVER (PARTITION BY base.session_id)) / NULLIF(AVG(cp.content_rate_avg) OVER (PARTITION BY base.session_id), 0) AS content_rate_avg_diff_percent,
        100.0 * (cp.content_review_count - AVG(cp.content_review_count) OVER (PARTITION BY base.session_id)) / NULLIF(AVG(cp.content_review_count) OVER (PARTITION BY base.session_id), 0) AS content_review_count_diff_percent,
        100.0 * (cp.content_review_wth_media_count - AVG(cp.content_review_wth_media_count) OVER (PARTITION BY base.session_id)) / NULLIF(AVG(cp.content_review_wth_media_count) OVER (PARTITION BY base.session_id), 0) AS content_review_wth_media_count_diff_percent,

        -- --- USER-CATEGORY AFFINITY FEATURES ---
        ucf.user_l1_cat_conversion_rate,
        ucf.user_l2_cat_conversion_rate,
        ucf.user_leaf_cat_conversion_rate,
        ucf.user_l1_cat_clicks,
        ucf.user_l1_cat_orders,

        -- --- CONTENT POPULARITY & VELOCITY ---
        cpf.all_time_total_order,
        cpf.all_time_total_click,
        cpf.all_time_total_cart,
        cpf.all_time_total_fav,
        cpf.order_velocity_7d_vs_30d,
        cpf.click_velocity_7d_vs_30d,
        cpf.all_time_click_to_order_rate,

        -- --- CATEGORY-RELATIVE POPULARITY ---
        ccrf.order_rank_in_leaf_category,
        ccrf.order_rank_in_l2_category,
        ccrf.order_rank_in_l1_category,

        -- --- CONTENT-AGE AFFINITY ---
        caf.avg_interaction_age_historical,
        caf.stddev_interaction_age_historical,
        
        -- CV Tags Word Count
        length(split(cm.cv_tags, ',')) AS cv_tags_word_count,

        -- Content-Gender Affinity
        cgf.male_clicks_historical, cgf.male_orders_historical,
        cgf.female_clicks_historical, cgf.female_orders_historical,
        COALESCE(cgf.female_orders_historical / NULLIF(cgf.female_clicks_historical, 0), 0) as female_conversion_rate_historical,
        COALESCE(cgf.male_orders_historical / NULLIF(cgf.male_clicks_historical, 0), 0) as male_conversion_rate_historical,

        -- Price vs Category Average
        (cp.selling_price - cat_price.avg_price_in_leaf_category) AS price_diff_from_category_avg,

        -- --- ORIGINAL HISTORICAL FEATURES ---
        usf.click_to_order_24h_user_sitewide_log, usf.click_to_cart_24h_user_sitewide_log, usf.cart_to_order_24h_user_sitewide_log, usf.click_to_order_72h_user_sitewide_log, usf.click_to_cart_72h_user_sitewide_log, usf.cart_to_order_72h_user_sitewide_log,
        usf2.search_ctr_24h_user_search_log, usf2.search_ctr_72h_user_search_log,
        um.* EXCLUDE (user_id_hashed),
        uttf.search_ctr_24h_user_top_terms_log, uttf.search_ctr_72h_user_top_terms_log,
        uff.click_to_order_24h_user_fashion_sitewide_log, uff.click_to_cart_24h_user_fashion_sitewide_log, uff.cart_to_order_24h_user_fashion_sitewide_log, uff.click_to_order_72h_user_fashion_sitewide_log, uff.click_to_cart_72h_user_fashion_sitewide_log, uff.cart_to_order_72h_user_fashion_sitewide_log,
        ufsl.rolling_total_search_click_24h AS rolling_total_search_click_24h_user_fashion_search_log, ufsl.rolling_total_search_click_72h AS rolling_total_search_click_72h_user_fashion_search_log,
        tsf.search_ctr_24h_term_search_log, tsf.search_ctr_72h_term_search_log,
        csf.search_ctr_3d_content_search_log, csf.search_ctr_7d_content_search_log,
        cttf.search_ctr_24h_content_top_terms_log, cttf.search_ctr_72h_content_top_terms_log,
        cswf.click_to_order_3d_content_sitewide_log, cswf.click_to_cart_3d_content_sitewide_log, cswf.cart_to_order_3d_content_sitewide_log, cswf.click_to_order_7d_content_sitewide_log, cswf.click_to_cart_7d_content_sitewide_log, cswf.cart_to_order_7d_content_sitewide_log,
        cm.* EXCLUDE (content_id_hashed, cv_tags), -- Exclude cv_tags as it's used in similarity
        cp.* EXCLUDE (content_id_hashed, update_date),
        ucas.total_click_so_far,
        ucas.total_cart_so_far,
        ucas.total_fav_so_far,
        ucas.total_order_so_far,
        (EXTRACT(EPOCH FROM base.ts_hour - COALESCE(ucas.last_click_ts_hour, base.ts_hour)))/3600.0 AS user_content_click_recency,
        (EXTRACT(EPOCH FROM base.ts_hour - COALESCE(ucas.last_cart_ts_hour, base.ts_hour)))/3600.0 AS user_content_cart_recency,
        (EXTRACT(EPOCH FROM base.ts_hour - COALESCE(ucas.last_fav_ts_hour, base.ts_hour)))/3600.0 AS user_content_fav_recency,
        (EXTRACT(EPOCH FROM base.ts_hour - COALESCE(ucas.last_order_ts_hour, base.ts_hour)))/3600.0 AS user_content_order_recency,
        (EXTRACT(EPOCH FROM base.date - cm.content_creation_date) / 86400.0) AS content_tenure_days

    FROM base
    -- Original ASOF and LEFT JOINs
    ASOF LEFT JOIN user_sitewide_features AS usf ON base.user_id_hashed = usf.user_id_hashed AND base.ts_hour > usf.ts_hour
    ASOF LEFT JOIN user_search_features AS usf2 ON base.user_id_hashed = usf2.user_id_hashed AND base.ts_hour > usf2.ts_hour
    ASOF LEFT JOIN user_top_terms_features AS uttf ON base.user_id_hashed = uttf.user_id_hashed AND base.search_term_normalized = uttf.search_term_normalized AND base.ts_hour > uttf.ts_hour
    ASOF LEFT JOIN user_fashion_sitewide_features AS uff ON base.user_id_hashed = uff.user_id_hashed AND base.content_id_hashed = uff.content_id_hashed AND base.ts_hour > uff.ts_hour
    ASOF LEFT JOIN user_fashion_search_log_rolled AS ufsl ON base.user_id_hashed = ufsl.user_id_hashed AND base.content_id_hashed = ufsl.content_id_hashed AND base.ts_hour > ufsl.ts_hour
    ASOF LEFT JOIN term_search_features AS tsf ON base.search_term_normalized = tsf.search_term_normalized AND base.ts_hour > tsf.ts_hour
    ASOF LEFT JOIN content_search_features AS csf ON base.content_id_hashed = csf.content_id_hashed AND base.date > csf.date
    ASOF LEFT JOIN content_top_terms_features AS cttf ON base.content_id_hashed = cttf.content_id_hashed AND base.search_term_normalized = cttf.search_term_normalized AND base.date > cttf.date
    ASOF LEFT JOIN content_sitewide_features AS cswf ON base.content_id_hashed = cswf.content_id_hashed AND base.date > cswf.date
    ASOF LEFT JOIN content_price AS cp ON base.content_id_hashed = cp.content_id_hashed AND base.date > cp.update_date
    ASOF LEFT JOIN user_content_action_stats AS ucas ON base.user_id_hashed = ucas.user_id_hashed AND base.content_id_hashed = ucas.content_id_hashed AND base.ts_hour > ucas.ts_hour
    LEFT JOIN user_metadata AS um ON base.user_id_hashed = um.user_id_hashed
    LEFT JOIN content_metadata AS cm ON base.content_id_hashed = cm.content_id_hashed
    ASOF LEFT JOIN user_category_features AS ucf ON base.user_id_hashed = ucf.user_id_hashed
        AND cm.level1_category_name = ucf.level1_category_name
        AND cm.level2_category_name = ucf.level2_category_name
        AND cm.leaf_category_name = ucf.leaf_category_name
        AND base.ts_hour > ucf.ts_hour

    -- ASOF JOINS for popularity features
    ASOF LEFT JOIN content_popularity_features AS cpf ON base.content_id_hashed = cpf.content_id_hashed AND base.date > cpf.date
    ASOF LEFT JOIN content_category_rank_features AS ccrf ON base.content_id_hashed = ccrf.content_id_hashed AND base.date > ccrf.date
    -- ASOF JOIN for content-age features
    ASOF LEFT JOIN content_age_features AS caf ON base.content_id_hashed = caf.content_id_hashed AND base.ts_hour > caf.ts_hour

    -- NEW ASOF JOINS
    ASOF LEFT JOIN content_gender_features AS cgf ON base.content_id_hashed = cgf.content_id_hashed AND base.ts_hour > cgf.ts_hour
    ASOF LEFT JOIN category_price_features AS cat_price ON cm.leaf_category_name = cat_price.leaf_category_name AND base.date > cat_price.update_date
    """

def generate_features(con):
    """
    Main function to connect to DuckDB, build the queries,
    execute them, and return the dataframes.
    """
    # --- Preprocessing User Metadata ---
    print("--- Preprocessing user_metadata.parquet ---")
    user_meta_df = duckdb.sql(f"SELECT * FROM read_parquet('{DATA_PATH}/user/metadata.parquet')").df()

    current_year = 2025
    min_birth_year = current_year - 90
    max_birth_year = current_year - 14

    valid_years_mask = user_meta_df['user_birth_year'].between(min_birth_year, max_birth_year)
    median_birth_year = user_meta_df.loc[valid_years_mask, 'user_birth_year'].median()

    if pd.isna(median_birth_year):
        median_birth_year = current_year - 30

    print(f"Calculated median birth year from valid entries: {int(median_birth_year)}")

    user_meta_df['user_birth_year'] = user_meta_df['user_birth_year'].apply(
        lambda x: x if pd.notna(x) and min_birth_year <= x <= max_birth_year else median_birth_year
    )
    user_meta_df['user_birth_year'] = user_meta_df['user_birth_year'].fillna(median_birth_year).astype(int)

    # Make the preprocessed dataframe available to the SQL query
    con.register('user_meta_df', user_meta_df)

    # --- Build and Execute the full TRAIN query ---
    train_query = f"""
    CREATE OR REPLACE TABLE final_train AS
    WITH
    {get_base_query('train_sessions.parquet')},
    {get_feature_queries()}
    {get_final_select_query()};
    """
    print("--- Generating Train DataFrame ---")
    con.execute(train_query)
    full_train_df = con.table('final_train').df()
    print("Train DataFrame generated. Shape:", full_train_df.shape)

    # --- Build and Execute the full TEST query ---
    test_query = f"""
    CREATE OR REPLACE TABLE final_test AS
    WITH
    {get_base_query('test_sessions.parquet')},
    {get_feature_queries()}
    {get_final_select_query()};
    """
    print("--- Generating Test DataFrame ---")
    con.execute(test_query)
    full_test_df = con.table('final_test').df()
    print("Test DataFrame generated. Shape:", full_test_df.shape)

    return full_train_df, full_test_df

### Preprocessing

In [ ]:
# --- 1. Feature Generation ---
# Connect to an in-memory DuckDB database
connection = duckdb.connect(database=':memory:', read_only=False)
train_df, test_df = generate_features(connection)
connection.close()

# --- 2. Target Definition ---
train_df["target"] = (
    train_df["clicked"] * 2.2 +
    train_df["added_to_fav"] * 0.1 +
    train_df["added_to_cart"] * 8.0 +
    train_df["ordered"] * 9.0
)

# --- 3. Feature Selection ---
# Define columns to be dropped initially
drop_cols = [
    "ordered", "clicked", "ts_hour", "date", "content_creation_date",
    'added_to_cart', 'added_to_fav', "session_id", "content_id_hashed",
    "user_id_hashed", "cv_tags", "search_term_normalized", "target"
]

# Statically define columns to drop due to high correlation
high_corr_drop_cols = [
    'user_birth_year', 'session_avg_rate_count', 'session_std_rate_count',
    'session_min_rate_count', 'session_max_rate_count', 'content_review_count_rank',
    'content_review_count_diff_percent', 'content_review_wth_media_count_diff_percent',
    'user_l1_cat_clicks', 'user_l1_cat_orders', 'search_ctr_24h_user_top_terms_log',
    'search_ctr_72h_user_top_terms_log', 'user_leaf_cat_conversion_rate',
    'search_ctr_7d_content_search_log', 'selling_price', 'discounted_price',
    'content_rate_count'
]
# Remove duplicates from the static list
high_corr_drop_cols = list(dict.fromkeys(high_corr_drop_cols))

print(f"Dropping {len(high_corr_drop_cols)} columns due to high correlation.")

# Determine the final list of features for training
features = [
    col for col in train_df.columns
    if col not in drop_cols and col not in high_corr_drop_cols
]

# --- 4. Prepare Data for CatBoost ---
X_train = train_df[features].copy()
y_train = train_df["target"].copy()
group_id_train = train_df["session_id"].copy()

X_test = test_df[features].copy()

# Process categorical features
cat_features = ["user_gender", "level1_category_name", "level2_category_name", "leaf_category_name"]
for col in cat_features:
    if col in X_train.columns:
        X_train[col] = X_train[col].fillna("UNKNOWN")
        X_test[col] = X_test[col].fillna("UNKNOWN")

# Sort data by session_id (required for CatBoost Ranker)
sorted_indices = group_id_train.argsort()
X_train_sorted = X_train.iloc[sorted_indices].reset_index(drop=True)
y_train_sorted = y_train.iloc[sorted_indices].reset_index(drop=True)
group_id_train_sorted = group_id_train.iloc[sorted_indices].reset_index(drop=True)

### Modelling and Prediction

In [ ]:
# --- 5. Model Training ---
print("\n--- Training CatBoostRanker on the full dataset ---")
ranker = CatBoostRanker(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    loss_function='YetiRank',
    random_seed=42,
    verbose=100,
    cat_features=cat_features
)

ranker.fit(
    X_train_sorted,
    y_train_sorted,
    group_id=group_id_train_sorted
)

# --- 6. Prediction and Submission ---
print("\n--- Generating predictions on the test set ---")

# Ensure test feature order matches training feature order
test_features_aligned = X_test[X_train_sorted.columns]

# Predict scores
test_predictions = ranker.predict(test_features_aligned)

# Add predictions to the original test dataframe
kaan_predictions = test_df.copy()
kaan_predictions["kaan_prediction"] = test_predictions
kaan_predictions = kaan_predictions[["session_id","user_id_hashed","content_id_hashed","kaan_prediction"]].copy()
kaan_predictions.to_csv("kaan_predictions.csv", index=False)

### Memory Cleaning

In [ ]:
del X_test
del X_train
del X_train_sorted
del test_df
del test_features_aligned
del train_df
del group_id_train
del group_id_train_sorted
del sorted_indices
del y_train
del y_train_sorted
del ranker
del test_predictions
del connection
del cat_features
del high_corr_drop_cols
del col
del drop_cols
del features

## 2nd Solution (Bilalcan)

### Preprocessing and Feature Engineering

In [ ]:
train = pl.scan_parquet(f"{DATA_PATH}/train_sessions.parquet")
test = pl.scan_parquet(f"{DATA_PATH}/test_sessions.parquet")

train = train.with_columns(
    pl.col("ts_hour").dt.truncate("1d").cast(pl.Datetime("ms")).alias("date")
)
test = test.with_columns(
    pl.col("ts_hour").dt.truncate("1d").cast(pl.Datetime("ms")).alias("date")
)

user_sitewide = pl.scan_parquet(f"{DATA_PATH}/user/sitewide_log.parquet")
user_search = pl.scan_parquet(f"{DATA_PATH}/user/search_log.parquet")
fashion_sitewide = pl.scan_parquet(f"{DATA_PATH}/user/fashion_sitewide_log.parquet")
fashion_search = pl.scan_parquet(f"{DATA_PATH}/user/fashion_search_log.parquet")
term_search = pl.scan_parquet(f"{DATA_PATH}/term/search_log.parquet")
user_top_terms = pl.scan_parquet(f"{DATA_PATH}/user/top_terms_log.parquet")
content_sitewide = pl.scan_parquet(f"{DATA_PATH}/content/sitewide_log.parquet")
content_search = pl.scan_parquet(f"{DATA_PATH}/content/search_log.parquet")
content_top_terms = pl.scan_parquet(f"{DATA_PATH}/content/top_terms_log.parquet")
content_metadata = pl.scan_parquet(f"{DATA_PATH}/content/metadata.parquet")
content_price = pl.scan_parquet(f"{DATA_PATH}/content/price_rate_review_data.parquet")
user_metadata = pl.scan_parquet(f"{DATA_PATH}/user/metadata.parquet")

In [ ]:
train_df1 = add_user_history(
    train,
    user_sitewide,
    user_col = "user_id_hashed",
    time_col = "ts_hour",
    interaction_cols = ["total_click", "total_fav", "total_cart", "total_order"],
    ratio_groups = [
        ("total_click", "total_order"), ("total_click", "total_cart"), 
        ("total_click", "total_fav"), ("total_cart", "total_order")],
    alias = "user_sitewide",
    weights = {"total_click": 0.204, "total_fav": 0.066, "total_cart": 0.254, "total_order": 0.476}
).collect()

test_df1 = add_user_history(
    test,
    user_sitewide,
    user_col = "user_id_hashed",
    time_col = "ts_hour",
    interaction_cols = ["total_click", "total_fav", "total_cart", "total_order"],
    ratio_groups = [
        ("total_click", "total_order"), ("total_click", "total_cart"), 
        ("total_click", "total_fav"), ("total_cart", "total_order")],
    alias = "user_sitewide",
    weights = {"total_click": 0.204, "total_fav": 0.066, "total_cart": 0.254, "total_order": 0.476}
).collect()

In [ ]:
train_df2 = add_user_history(
    train,
    user_search,
    user_col = "user_id_hashed",
    time_col = "ts_hour",
    interaction_cols = ["total_search_impression", "total_search_click"],
    ratio_groups = [
        ("total_search_impression", "total_search_click")],
    alias = "user_search",
    weights = {"total_search_impression": 0.1, "total_search_click": 0.9}
).collect()

test_df2 = add_user_history(
    test,
    user_search,
    user_col = "user_id_hashed",
    time_col = "ts_hour",
    interaction_cols = ["total_search_impression", "total_search_click"],
    ratio_groups = [
        ("total_search_impression", "total_search_click")],
    alias = "user_search",
    weights = {"total_search_impression": 0.1, "total_search_click": 0.9}
).collect()

In [ ]:
train_df3 = add_decay_features_multiple(
    train,
    fashion_sitewide,
    rolling_windows=[3,12],
    interaction_cols=[
        "total_click","total_order","total_cart","total_fav"
    ],
    alias="fashion_site"
).collect()

test_df3 = add_decay_features_multiple(
    test,
    fashion_sitewide,
    rolling_windows=[3,12],
    interaction_cols=[
        "total_click","total_order","total_cart","total_fav"
    ],
    alias="fashion_site"
).collect()

In [ ]:
train_df4 = add_decay_features_multiple(
    train,
    fashion_search,
    rolling_windows=[3,12],
    interaction_cols=[
        "total_search_click","total_search_impression"
    ],
    alias="fashion_search"
).collect()

test_df4 = add_decay_features_multiple(
    test,
    fashion_search,
    rolling_windows=[3,12],
    interaction_cols=[
        "total_search_click","total_search_impression"
    ],
    alias="fashion_search"
).collect()

In [ ]:
train_df5 = add_user_history(
    df = train,
    user_df = term_search,
    user_col = "search_term_normalized",
    time_col = "ts_hour",
    interaction_cols = ["total_search_click", "total_search_impression"],
    ratio_groups = [("total_search_impression", "total_search_click")],
    alias = "term_search",
    weights = {"total_search_click": 0.9, "total_search_impression": 0.1}
).collect()

test_df5 = add_user_history(
    df = test,
    user_df = term_search,
    user_col = "search_term_normalized",
    time_col = "ts_hour",
    interaction_cols = ["total_search_click", "total_search_impression"],
    ratio_groups = [("total_search_impression", "total_search_click")],
    alias = "term_search",
    weights = {"total_search_click": 0.9, "total_search_impression": 0.1}
).collect()

In [ ]:
train_df6 = add_user_term_history(
    df = train,
    user_df = user_top_terms,
    user_col = "user_id_hashed",
    time_col = "ts_hour",
    term_col = "search_term_normalized",
    interaction_cols = ["total_search_impression","total_search_click"],
    alias = "user_top_terms",
    ratio_groups = [("total_search_impression", "total_search_click")],
    weights = {"total_search_click": 0.9, "total_search_impression": 0.1}
).collect()

test_df6 = add_user_term_history(
    df = test,
    user_df = user_top_terms,
    user_col = "user_id_hashed",
    time_col = "ts_hour",
    term_col = "search_term_normalized",
    interaction_cols = ["total_search_impression","total_search_click"],
    alias = "user_top_terms",
    ratio_groups = [("total_search_impression", "total_search_click")],
    weights = {"total_search_click": 0.9, "total_search_impression": 0.1}
).collect()

In [ ]:
train_df7 = add_user_history(
    train,
    content_sitewide,
    user_col = "content_id_hashed",
    time_col = "date",
    interaction_cols = ["total_click", "total_fav", "total_cart", "total_order"],
    ratio_groups = [
        ("total_click", "total_order"), ("total_click", "total_cart"), 
        ("total_click", "total_fav"), ("total_cart", "total_order")],
    alias = "content_sitewide",
    weights = {"total_click": 0.204, "total_fav": 0.066, "total_cart": 0.254, "total_order": 0.476}
).collect()

test_df7 = add_user_history(
    test,
    content_sitewide,
    user_col = "content_id_hashed",
    time_col = "date",
    interaction_cols = ["total_click", "total_fav", "total_cart", "total_order"],
    ratio_groups = [
        ("total_click", "total_order"), ("total_click", "total_cart"), 
        ("total_click", "total_fav"), ("total_cart", "total_order")],
    alias = "content_sitewide",
    weights = {"total_click": 0.204, "total_fav": 0.066, "total_cart": 0.254, "total_order": 0.476}
).collect()

In [ ]:
train_df8 = add_user_history(
    train,
    content_search,
    user_col = "content_id_hashed",
    time_col = "date",
    interaction_cols = ["total_search_impression", "total_search_click"],
    ratio_groups = [
        ("total_search_impression", "total_search_click")],
    alias = "content_search",
    weights = {"total_search_impression": 0.1, "total_search_click": 0.9}
).collect()

test_df8 = add_user_history(
    test,
    content_search,
    user_col = "content_id_hashed",
    time_col = "date",
    interaction_cols = ["total_search_impression", "total_search_click"],
    ratio_groups = [
        ("total_search_impression", "total_search_click")],
    alias = "content_search",
    weights = {"total_search_impression": 0.1, "total_search_click": 0.9}
).collect()

In [ ]:
train_df9 = add_user_term_history(
    df = train,
    user_df = content_top_terms,
    user_col = "content_id_hashed",
    time_col = "date",
    term_col = "search_term_normalized",
    interaction_cols = ["total_search_impression","total_search_click"],
    alias = "content_top_terms",
    ratio_groups = [("total_search_impression", "total_search_click")],
    weights = {"total_search_click": 0.9, "total_search_impression": 0.1}
).collect()

test_df9 = add_user_term_history(
    df = test,
    user_df = content_top_terms,
    user_col = "content_id_hashed",
    time_col = "date",
    term_col = "search_term_normalized",
    interaction_cols = ["total_search_impression","total_search_click"],
    alias = "content_top_terms",
    ratio_groups = [("total_search_impression", "total_search_click")],
    weights = {"total_search_click": 0.9, "total_search_impression": 0.1}
).collect()

In [ ]:
train_df10 = add_content_price_history(
    df=train,
    content_price=content_price,
    content_metadata=content_metadata,
    content_col="content_id_hashed",
    left_time_col="date",
    right_time_col="update_date",
    categories=["level1_category_name", "level2_category_name", "leaf_category_name"],
    bayesian_m=30,
    psuedo_alpha=1,
    psuedo_beta=1,
    wilson_z=1.96,
    exact_match=True
).collect()

test_df10 = add_content_price_history(
    df=test,
    content_price=content_price,
    content_metadata=content_metadata,
    content_col="content_id_hashed",
    left_time_col="date",
    right_time_col="update_date",
    categories=["level1_category_name", "level2_category_name", "leaf_category_name"],
    bayesian_m=30,
    psuedo_alpha=1,
    psuedo_beta=1,
    wilson_z=1.96,
    exact_match=True
).collect()

In [ ]:
train_df11 = add_user_metadata(
    df=train,
    user_metadata=user_metadata,
    user_col="user_id_hashed"
).collect()

test_df11 = add_user_metadata(
    df=test,
    user_metadata=user_metadata,
    user_col="user_id_hashed"
).collect()

In [ ]:
train_df12 = add_time_history(
    df = train,
    df_value = user_sitewide,
    key_col = "user_id_hashed",
    index_col = "ts_hour",
    periods = ["24h","72h"],
    cols = ["total_click", "total_order", "total_cart", "total_fav"],
    ratio_cols = [("total_click", "total_order"), ("total_cart", "total_order"), ("total_fav", "total_order"), ("total_click", "total_cart"), ("total_click", "total_fav")],
    aggs = ["mean","std","min","max","sum"], 
    ratio_aggs = ["mean","std","sum"],
    alias = "user_sitewide",
    exact_match = True
).collect()

test_df12 = add_time_history(
    df = test,
    df_value = user_sitewide,
    key_col = "user_id_hashed",
    index_col = "ts_hour",
    periods = ["24h","72h"],
    cols = ["total_click", "total_order", "total_cart", "total_fav"],
    ratio_cols = [("total_click", "total_order"), ("total_cart", "total_order"), ("total_fav", "total_order"), ("total_click", "total_cart"), ("total_click", "total_fav")],
    aggs = ["mean","std","min","max","sum"], 
    ratio_aggs = ["mean","std","sum"],
    alias = "user_sitewide",
    exact_match = True
).collect()

In [ ]:
train_df13 = add_time_history(
    df = train,
    df_value = user_search,
    key_col = "user_id_hashed",
    index_col = "ts_hour",
    periods = ["24h","72h"],
    cols = ["total_search_impression", "total_search_click"],
    ratio_cols = [("total_search_impression", "total_search_click")],
    aggs = ["mean","std","min","max","sum"], 
    ratio_aggs = ["mean","std","sum"],
    alias = "user_search",
    exact_match = True
).collect()

test_df13 = add_time_history(
    df = test,
    df_value = user_search,
    key_col = "user_id_hashed",
    index_col = "ts_hour",
    periods = ["24h","72h"],
    cols = ["total_search_impression", "total_search_click"],
    ratio_cols = [("total_search_impression", "total_search_click")],
    aggs = ["mean","std","min","max","sum"], 
    ratio_aggs = ["mean","std","sum"],
    alias = "user_search",
    exact_match = True
).collect()

In [ ]:
train_df14 = add_time_history(
    df = train,
    df_value = content_sitewide,
    key_col = "content_id_hashed",
    index_col = "date",
    periods = ["3d","7d"],
    cols = ["total_click", "total_order", "total_cart", "total_fav"],
    ratio_cols = [("total_click", "total_order"), ("total_cart", "total_order"), ("total_fav", "total_order"), ("total_click", "total_cart"), ("total_click", "total_fav")],
    aggs = ["mean","std","min","max","sum"],
    ratio_aggs = ["mean","std","sum"],
    alias = "content_sitewide",
    exact_match = True
).collect()

test_df14 = add_time_history(
    df = test,
    df_value = content_sitewide,
    key_col = "content_id_hashed",
    index_col = "date",
    periods = ["3d","7d"],
    cols = ["total_click", "total_order", "total_cart", "total_fav"],
    ratio_cols = [("total_click", "total_order"), ("total_cart", "total_order"), ("total_fav", "total_order"), ("total_click", "total_cart"), ("total_click", "total_fav")],
    aggs = ["mean","std","min","max","sum"],
    ratio_aggs = ["mean","std","sum"],
    alias = "content_sitewide",
    exact_match = True
).collect()

In [ ]:
train_df15 = add_time_history(
    df = train,
    df_value = content_search,
    key_col = "content_id_hashed",
    index_col = "date",
    periods = ["3d","7d"],
    cols = ["total_search_impression", "total_search_click"],
    ratio_cols = [("total_search_impression", "total_search_click")],
    aggs = ["mean","std","min","max","sum"],
    ratio_aggs = ["mean","std","sum"],
    alias = "content_search",
    exact_match = True
).collect()

test_df15 = add_time_history(
    df = test,
    df_value = content_search,
    key_col = "content_id_hashed",
    index_col = "date",
    periods = ["3d","7d"],
    cols = ["total_search_impression", "total_search_click"],
    ratio_cols = [("total_search_impression", "total_search_click")],
    aggs = ["mean","std","min","max","sum"],
    ratio_aggs = ["mean","std","sum"],
    alias = "content_search",
    exact_match = True
).collect()

In [ ]:
base_cols = ["ts_hour", "date", "search_term_normalized", "clicked", "ordered", "added_to_cart", "added_to_fav", "user_id_hashed", "content_id_hashed", "session_id"]
train = train.collect().join(train_df1, on=base_cols, how="left")
train = train.join(train_df2, on=base_cols, how="left")
train = train.join(train_df3, on=base_cols, how="left")
train = train.join(train_df4, on=base_cols, how="left")
train = train.join(train_df5, on=base_cols, how="left")
train = train.join(train_df6, on=base_cols, how="left")
train = train.join(train_df7, on=base_cols, how="left")
train = train.join(train_df8, on=base_cols, how="left")
train = train.join(train_df9, on=base_cols, how="left")
train = train.join(train_df10, on=base_cols, how="left")
train = train.join(train_df11, on=base_cols, how="left")
train = train.join(train_df12, on=base_cols, how="left")
train = train.join(train_df13, on=base_cols, how="left")
train = train.join(train_df14, on=base_cols, how="left")
train = train.join(train_df15, on=base_cols, how="left")

In [ ]:
base_cols = ["ts_hour", "date", "search_term_normalized", "user_id_hashed", "content_id_hashed", "session_id"]
test = test.collect().join(test_df1, on=base_cols, how="left")
test = test.join(test_df2, on=base_cols, how="left")
test = test.join(test_df3, on=base_cols, how="left")
test = test.join(test_df4, on=base_cols, how="left")
test = test.join(test_df5, on=base_cols, how="left")
test = test.join(test_df6, on=base_cols, how="left")
test = test.join(test_df7, on=base_cols, how="left")
test = test.join(test_df8, on=base_cols, how="left")
test = test.join(test_df9, on=base_cols, how="left")
test = test.join(test_df10, on=base_cols, how="left")
test = test.join(test_df11, on=base_cols, how="left")
test = test.join(test_df12, on=base_cols, how="left")
test = test.join(test_df13, on=base_cols, how="left")
test = test.join(test_df14, on=base_cols, how="left")
test = test.join(test_df15, on=base_cols, how="left")

In [ ]:
train = candidate_counter(
    train,
    session_col = "session_id",
    content_col = "content_id_hashed"
)

test = candidate_counter(
    test,
    session_col = "session_id",
    content_col = "content_id_hashed"
)

In [ ]:
session_col = "session_id"
sitewide_table = "content_sitewide"
tables = ["content_top_terms","content_sitewide","content_search"]
cols_search = ["total_search_impression","total_search_click"]
cols_sitewide = ["total_click","total_cart","total_order","total_fav"]
avg_ratio_template = "{table}_{col1}_to_{col2}_avg_ratio"
non_agg_col_template = "{table}_{col}"
weighted_col_template = "{table}_weighted_score"
weights = {
    "total_order": 0.476,
    "total_click": 0.204,
    "total_cart": 0.254,
    "total_fav": 0.066,
    "total_search_impression": 0.1,
    "total_search_click": 0.9
}
table_weights = {
    "content_top_terms": 0.3,
    "content_sitewide": 0.6,
    "content_search": 0.1
}
price_columns = ["original_price","selling_price","discounted_price"]

train = session_based_ranking_for_contents(
    train,
    session_col = session_col,
    sitewide_table = sitewide_table,
    tables = tables,
    cols_search = cols_search,
    cols_sitewide = cols_sitewide,
    avg_ratio_template = avg_ratio_template,
    non_agg_col_template = non_agg_col_template,
    weighted_col_template = weighted_col_template,
    weights = weights,
    table_weights = table_weights,
    price_columns = price_columns
)

test = session_based_ranking_for_contents(
    test,
    session_col = session_col,
    sitewide_table = sitewide_table,
    tables = tables,
    cols_search = cols_search,
    cols_sitewide = cols_sitewide,
    avg_ratio_template = avg_ratio_template,
    non_agg_col_template = non_agg_col_template,
    weighted_col_template = weighted_col_template,
    weights = weights,
    table_weights = table_weights,
    price_columns = price_columns
)

### Memory Cleaning

In [ ]:
del train_df1
del train_df2
del train_df3
del train_df4
del train_df5
del train_df6
del train_df7
del train_df8
del train_df9
del train_df10
del train_df11
del train_df12
del train_df13
del train_df14
del train_df15
del test_df1
del test_df2
del test_df3
del test_df4
del test_df5
del test_df6
del test_df7
del test_df8
del test_df9
del test_df10
del test_df11
del test_df12
del test_df13
del test_df14
del test_df15

### Modelling

#### Model Settings

In [ ]:
features = [
    "user_sitewide_total_click",
    "user_sitewide_total_cart",
    "user_sitewide_total_fav",
    "user_sitewide_total_order",
    "user_sitewide_total_click_sum",
    "user_sitewide_total_fav_sum",
    "user_sitewide_total_cart_sum",
    "user_sitewide_total_order_sum",
    "user_sitewide_total_click_max",
    "user_sitewide_total_fav_max",
    "user_sitewide_total_cart_max",
    "user_sitewide_total_order_max",
    "user_sitewide_total_click_std",
    "user_sitewide_total_fav_std",
    "user_sitewide_total_cart_std",
    "user_sitewide_total_order_std",
    "user_sitewide_total_fav_active_session_count",
    "user_sitewide_total_cart_active_session_count",
    "user_sitewide_total_order_active_session_count",
    "user_sitewide_session_count",
    "user_sitewide_total_click_avg",
    "user_sitewide_total_fav_avg",
    "user_sitewide_total_cart_avg",
    "user_sitewide_total_order_avg",
    "user_sitewide_total_click_active_session_ratio",
    "user_sitewide_total_fav_active_session_ratio",
    "user_sitewide_total_cart_active_session_ratio",
    "user_sitewide_total_order_active_session_ratio",
    "user_sitewide_total_click_to_total_order_avg_ratio",
    "user_sitewide_total_click_to_total_cart_avg_ratio",
    "user_sitewide_total_click_to_total_fav_avg_ratio",
    "user_sitewide_total_cart_to_total_order_avg_ratio",
    "user_sitewide_weighted_sum_score",
    "user_sitewide_weighted_avg_score",
    "user_search_total_search_impression",
    "user_search_total_search_click",
    "user_search_total_search_impression_sum",
    "user_search_total_search_click_sum",
    "user_search_total_search_impression_max",
    "user_search_total_search_click_max",
    "user_search_total_search_impression_std",
    "user_search_total_search_click_std",
    "user_search_total_search_impression_active_session_count",
    "user_search_total_search_click_active_session_count",
    "user_search_total_search_impression_avg",
    "user_search_total_search_click_avg",
    "user_search_total_search_impression_active_session_ratio",
    "user_search_total_search_click_active_session_ratio",
    "user_search_total_search_impression_to_total_search_click_avg_ratio",
    "user_search_weighted_sum_score",
    "user_search_weighted_avg_score",
    "total_click_decay_score_fashion_site",
    "total_order_decay_score_fashion_site",
    "total_cart_decay_score_fashion_site",
    "total_fav_decay_score_fashion_site",
    "total_click_weighted_3roll_step_mean_fashion_site",
    "total_click_weighted_3roll_step_std_fashion_site",
    "total_click_3roll_step_mean_fashion_site",
    "total_click_3roll_step_sum_fashion_site",
    "total_click_weighted_12roll_step_std_fashion_site",
    "total_click_12roll_step_sum_fashion_site",
    "total_order_weighted_3roll_step_mean_fashion_site",
    "total_order_weighted_3roll_step_std_fashion_site",
    "total_order_weighted_12roll_step_mean_fashion_site",
    "total_order_12roll_step_mean_fashion_site",
    "total_cart_weighted_3roll_step_mean_fashion_site",
    "total_cart_weighted_3roll_step_std_fashion_site",
    "total_cart_3roll_step_mean_fashion_site",
    "total_cart_weighted_12roll_step_mean_fashion_site",
    "total_cart_weighted_12roll_step_std_fashion_site",
    "total_cart_12roll_step_mean_fashion_site",
    "total_cart_12roll_step_sum_fashion_site",
    "total_fav_weighted_3roll_step_std_fashion_site",
    "total_fav_weighted_12roll_step_mean_fashion_site",
    "total_fav_weighted_12roll_step_std_fashion_site",
    "total_fav_12roll_step_mean_fashion_site",
    "total_click_fashion_site",
    "total_order_fashion_site",
    "total_cart_fashion_site",
    "total_fav_fashion_site",
    "total_search_click_decay_score_fashion_search",
    "total_search_impression_decay_score_fashion_search",
    "total_search_click_weighted_3roll_step_mean_fashion_search",
    "total_search_click_weighted_3roll_step_std_fashion_search",
    "total_search_click_3roll_step_mean_fashion_search",
    "total_search_click_3roll_step_sum_fashion_search",
    "total_search_click_weighted_12roll_step_std_fashion_search",
    "total_search_click_12roll_step_mean_fashion_search",
    "total_search_click_12roll_step_sum_fashion_search",
    "total_search_impression_weighted_3roll_step_mean_fashion_search",
    "total_search_impression_weighted_3roll_step_std_fashion_search",
    "total_search_impression_3roll_step_sum_fashion_search",
    "total_search_impression_weighted_12roll_step_std_fashion_search",
    "total_search_impression_12roll_step_sum_fashion_search",
    "total_search_click_fashion_search",
    "total_search_impression_fashion_search",
    "term_search_total_search_impression",
    "term_search_total_search_click",
    "term_search_total_search_click_max",
    "term_search_total_search_impression_max",
    "term_search_total_search_click_std",
    "term_search_total_search_impression_std",
    "term_search_total_search_impression_active_session_count",
    "term_search_total_search_click_avg",
    "term_search_total_search_click_active_session_ratio",
    "term_search_total_search_impression_active_session_ratio",
    "term_search_total_search_impression_to_total_search_click_avg_ratio",
    "user_top_terms_total_search_impression",
    "user_top_terms_total_search_click",
    "user_top_terms_total_search_impression_sum",
    "user_top_terms_total_search_click_sum",
    "user_top_terms_total_search_impression_max",
    "user_top_terms_total_search_click_max",
    "user_top_terms_total_search_impression_std",
    "user_top_terms_total_search_click_std",
    "user_top_terms_total_search_impression_active_session_count",
    "user_top_terms_total_search_impression_avg",
    "user_top_terms_total_search_click_avg",
    "user_top_terms_total_search_impression_to_total_search_click_avg_ratio",
    "user_top_terms_weighted_avg_score",
    "content_sitewide_total_click",
    "content_sitewide_total_cart",
    "content_sitewide_total_fav",
    "content_sitewide_total_order",
    "content_sitewide_total_fav_sum",
    "content_sitewide_total_cart_sum",
    "content_sitewide_total_click_max",
    "content_sitewide_total_fav_max",
    "content_sitewide_total_cart_max",
    "content_sitewide_total_order_max",
    "content_sitewide_total_click_std",
    "content_sitewide_total_fav_std",
    "content_sitewide_total_cart_std",
    "content_sitewide_total_order_std",
    "content_sitewide_total_click_active_session_count",
    "content_sitewide_total_fav_active_session_count",
    "content_sitewide_total_cart_active_session_count",
    "content_sitewide_total_order_active_session_count",
    "content_sitewide_total_fav_avg",
    "content_sitewide_total_cart_avg",
    "content_sitewide_total_order_avg",
    "content_sitewide_total_click_active_session_ratio",
    "content_sitewide_total_fav_active_session_ratio",
    "content_sitewide_total_cart_active_session_ratio",
    "content_sitewide_total_order_active_session_ratio",
    "content_sitewide_total_click_to_total_order_avg_ratio",
    "content_sitewide_total_click_to_total_cart_avg_ratio",
    "content_sitewide_total_click_to_total_fav_avg_ratio",
    "content_sitewide_total_cart_to_total_order_avg_ratio",
    "content_sitewide_weighted_avg_score",
    "content_search_total_search_impression",
    "content_search_total_search_click",
    "content_search_total_search_impression_sum",
    "content_search_total_search_click_sum",
    "content_search_total_search_impression_max",
    "content_search_total_search_click_max",
    "content_search_total_search_impression_std",
    "content_search_total_search_click_std",
    "content_search_total_search_impression_active_session_count",
    "content_search_total_search_click_active_session_count",
    "content_search_total_search_impression_avg",
    "content_search_total_search_click_avg",
    "content_search_total_search_click_active_session_ratio",
    "content_search_total_search_impression_to_total_search_click_avg_ratio",
    "content_search_weighted_sum_score",
    "content_search_weighted_avg_score",
    "content_top_terms_total_search_impression",
    "content_top_terms_total_search_click",
    "content_top_terms_total_search_impression_sum",
    "content_top_terms_total_search_click_sum",
    "content_top_terms_total_search_impression_max",
    "content_top_terms_total_search_click_max",
    "content_top_terms_total_search_impression_std",
    "content_top_terms_total_search_click_std",
    "content_top_terms_total_search_impression_active_session_count",
    "content_top_terms_total_search_click_active_session_count",
    "content_top_terms_total_search_impression_avg",
    "content_top_terms_total_search_click_avg",
    "content_top_terms_total_search_click_active_session_ratio",
    "content_top_terms_total_search_impression_to_total_search_click_avg_ratio",
    "content_top_terms_weighted_sum_score",
    "content_top_terms_weighted_avg_score",
    "content_rate_count",
    "content_rate_avg",
    "level1_category_name",
    "level2_category_name",
    "leaf_category_name",
    "total_attribute_option_count",
    "merchant_count",
    "filterable_label_count",
    "level2_category_name_size",
    "leaf_category_name_size",
    "discount_rate",
    "selling_rate",
    "selling_discount_diff_ratio",
    "content_rate_avg_bayesian",
    "content_rate_to_review_smoothed_ratio",
    "content_review_to_media_smoothed_ratio",
    "wilson_score_review_to_media",
    "level2_category_name_selling_price_log_mean",
    "leaf_category_name_discounted_price_log_mean",
    "leaf_category_name_selling_price_log_std",
    "smoothed_leaf_category_name_selling_price_mean",
    "smoothed_leaf_category_name_selling_price_std",
    "rank_level1_category_name_original_price_log",
    "rank_level2_category_name_original_price_log",
    "rank_leaf_category_name_original_price_log",
    "rank_level1_category_name_selling_price_log",
    "rank_level2_category_name_selling_price_log",
    "rank_leaf_category_name_selling_price_log",
    "rank_level1_category_name_discounted_price_log",
    "rank_level2_category_name_discounted_price_log",
    "rank_leaf_category_name_discounted_price_log",
    "rank_level1_category_name_discount_rate",
    "rank_level2_category_name_discount_rate",
    "rank_leaf_category_name_discount_rate",
    "rank_level1_category_name_selling_rate",
    "rank_level2_category_name_selling_rate",
    "rank_leaf_category_name_selling_rate",
    "rank_level1_category_name_content_rate_avg_bayesian",
    "rank_level2_category_name_content_rate_avg_bayesian",
    "rank_leaf_category_name_content_rate_avg_bayesian",
    "rank_level1_category_name_content_review_count_norm",
    "rank_level2_category_name_content_review_count_norm",
    "rank_leaf_category_name_content_review_count_norm",
    "rank_level1_category_name_content_review_wth_media_count_norm",
    "rank_level2_category_name_content_review_wth_media_count_norm",
    "rank_leaf_category_name_content_review_wth_media_count_norm",
    "rank_level1_category_name_wilson_score_rate_to_review",
    "rank_level2_category_name_wilson_score_rate_to_review",
    "rank_leaf_category_name_wilson_score_rate_to_review",
    "rank_level1_category_name_wilson_score_review_to_media",
    "rank_level2_category_name_wilson_score_review_to_media",
    "rank_leaf_category_name_wilson_score_review_to_media",
    "content_tenure_hours",
    "update_to_content_tenure_ratio",
    "user_gender",
    "user_age",
    "user_sign_up_age",
    "user_tenure_in_days",
    "user_sitewide_rolling_mean_total_click_72h",
    "user_sitewide_rolling_std_total_click_72h",
    "user_sitewide_rolling_min_total_click_72h",
    "user_sitewide_rolling_max_total_click_72h",
    "user_sitewide_rolling_sum_total_click_72h",
    "user_sitewide_rolling_mean_total_order_72h",
    "user_sitewide_rolling_std_total_order_72h",
    "user_sitewide_rolling_max_total_order_72h",
    "user_sitewide_rolling_sum_total_order_72h",
    "user_sitewide_rolling_mean_total_cart_72h",
    "user_sitewide_rolling_std_total_cart_72h",
    "user_sitewide_rolling_max_total_cart_72h",
    "user_sitewide_rolling_sum_total_cart_72h",
    "user_sitewide_rolling_mean_total_fav_72h",
    "user_sitewide_rolling_std_total_fav_72h",
    "user_sitewide_rolling_max_total_fav_72h",
    "user_sitewide_rolling_sum_total_fav_72h",
    "user_sitewide_rolling_mean_total_click_24h",
    "user_sitewide_rolling_std_total_click_24h",
    "user_sitewide_rolling_min_total_click_24h",
    "user_sitewide_rolling_max_total_click_24h",
    "user_sitewide_rolling_sum_total_click_24h",
    "user_sitewide_rolling_mean_total_order_24h",
    "user_sitewide_rolling_std_total_order_24h",
    "user_sitewide_rolling_min_total_order_24h",
    "user_sitewide_rolling_mean_total_cart_24h",
    "user_sitewide_rolling_std_total_cart_24h",
    "user_sitewide_rolling_min_total_cart_24h",
    "user_sitewide_rolling_max_total_cart_24h",
    "user_sitewide_rolling_sum_total_cart_24h",
    "user_sitewide_rolling_mean_total_fav_24h",
    "user_sitewide_rolling_std_total_fav_24h",
    "user_sitewide_rolling_min_total_fav_24h",
    "user_sitewide_rolling_max_total_fav_24h",
    "user_sitewide_rolling_sum_total_fav_24h",
    "user_sitewide_total_click_to_total_order_mean_24h_ratio",
    "user_sitewide_total_cart_to_total_order_mean_24h_ratio",
    "user_sitewide_total_fav_to_total_order_mean_24h_ratio",
    "user_sitewide_total_click_to_total_cart_mean_24h_ratio",
    "user_sitewide_total_click_to_total_fav_mean_24h_ratio",
    "user_sitewide_total_click_to_total_order_std_24h_ratio",
    "user_sitewide_total_cart_to_total_order_std_24h_ratio",
    "user_sitewide_total_fav_to_total_order_std_24h_ratio",
    "user_sitewide_total_click_to_total_cart_std_24h_ratio",
    "user_sitewide_total_click_to_total_fav_std_24h_ratio",
    "user_sitewide_total_click_to_total_cart_sum_24h_ratio",
    "user_sitewide_total_click_to_total_order_mean_72h_ratio",
    "user_sitewide_total_cart_to_total_order_mean_72h_ratio",
    "user_sitewide_total_fav_to_total_order_mean_72h_ratio",
    "user_sitewide_total_click_to_total_cart_mean_72h_ratio",
    "user_sitewide_total_click_to_total_fav_mean_72h_ratio",
    "user_sitewide_total_click_to_total_order_std_72h_ratio",
    "user_sitewide_total_cart_to_total_order_std_72h_ratio",
    "user_sitewide_total_fav_to_total_order_std_72h_ratio",
    "user_sitewide_total_click_to_total_cart_std_72h_ratio",
    "user_sitewide_total_click_to_total_fav_std_72h_ratio",
    "user_sitewide_total_click_to_total_cart_sum_72h_ratio",
    "user_sitewide_total_click_lag1",
    "user_sitewide_total_order_lag1",
    "user_sitewide_total_cart_lag1",
    "user_sitewide_total_fav_lag1",
    "user_search_rolling_mean_total_search_impression_72h",
    "user_search_rolling_std_total_search_impression_72h",
    "user_search_rolling_min_total_search_impression_72h",
    "user_search_rolling_max_total_search_impression_72h",
    "user_search_rolling_sum_total_search_impression_72h",
    "user_search_rolling_mean_total_search_click_72h",
    "user_search_rolling_std_total_search_click_72h",
    "user_search_rolling_min_total_search_click_72h",
    "user_search_rolling_max_total_search_click_72h",
    "user_search_rolling_sum_total_search_click_72h",
    "user_search_rolling_mean_total_search_impression_24h",
    "user_search_rolling_std_total_search_impression_24h",
    "user_search_rolling_min_total_search_impression_24h",
    "user_search_rolling_max_total_search_impression_24h",
    "user_search_rolling_sum_total_search_impression_24h",
    "user_search_rolling_mean_total_search_click_24h",
    "user_search_rolling_std_total_search_click_24h",
    "user_search_rolling_min_total_search_click_24h",
    "user_search_rolling_max_total_search_click_24h",
    "user_search_rolling_sum_total_search_click_24h",
    "user_search_total_search_impression_to_total_search_click_mean_24h_ratio",
    "user_search_total_search_impression_to_total_search_click_std_24h_ratio",
    "user_search_total_search_impression_to_total_search_click_sum_24h_ratio",
    "user_search_total_search_impression_to_total_search_click_mean_72h_ratio",
    "user_search_total_search_impression_to_total_search_click_std_72h_ratio",
    "user_search_total_search_impression_to_total_search_click_sum_72h_ratio",
    "user_search_total_search_impression_lag1",
    "user_search_total_search_click_lag1",
    "content_sitewide_rolling_mean_total_click_7d",
    "content_sitewide_rolling_std_total_click_7d",
    "content_sitewide_rolling_min_total_click_7d",
    "content_sitewide_rolling_max_total_click_7d",
    "content_sitewide_rolling_sum_total_click_7d",
    "content_sitewide_rolling_mean_total_order_7d",
    "content_sitewide_rolling_std_total_order_7d",
    "content_sitewide_rolling_min_total_order_7d",
    "content_sitewide_rolling_max_total_order_7d",
    "content_sitewide_rolling_sum_total_order_7d",
    "content_sitewide_rolling_mean_total_cart_7d",
    "content_sitewide_rolling_std_total_cart_7d",
    "content_sitewide_rolling_min_total_cart_7d",
    "content_sitewide_rolling_max_total_cart_7d",
    "content_sitewide_rolling_mean_total_fav_7d",
    "content_sitewide_rolling_std_total_fav_7d",
    "content_sitewide_rolling_min_total_fav_7d",
    "content_sitewide_rolling_max_total_fav_7d",
    "content_sitewide_rolling_sum_total_fav_7d",
    "content_sitewide_rolling_mean_total_click_3d",
    "content_sitewide_rolling_std_total_click_3d",
    "content_sitewide_rolling_min_total_click_3d",
    "content_sitewide_rolling_max_total_click_3d",
    "content_sitewide_rolling_mean_total_order_3d",
    "content_sitewide_rolling_std_total_order_3d",
    "content_sitewide_rolling_min_total_order_3d",
    "content_sitewide_rolling_max_total_order_3d",
    "content_sitewide_rolling_mean_total_cart_3d",
    "content_sitewide_rolling_std_total_cart_3d",
    "content_sitewide_rolling_min_total_cart_3d",
    "content_sitewide_rolling_max_total_cart_3d",
    "content_sitewide_rolling_sum_total_cart_3d",
    "content_sitewide_rolling_mean_total_fav_3d",
    "content_sitewide_rolling_std_total_fav_3d",
    "content_sitewide_rolling_min_total_fav_3d",
    "content_sitewide_rolling_max_total_fav_3d",
    "content_sitewide_total_click_to_total_order_mean_3d_ratio",
    "content_sitewide_total_cart_to_total_order_mean_3d_ratio",
    "content_sitewide_total_fav_to_total_order_mean_3d_ratio",
    "content_sitewide_total_click_to_total_cart_mean_3d_ratio",
    "content_sitewide_total_click_to_total_fav_mean_3d_ratio",
    "content_sitewide_total_click_to_total_order_std_3d_ratio",
    "content_sitewide_total_cart_to_total_order_std_3d_ratio",
    "content_sitewide_total_fav_to_total_order_std_3d_ratio",
    "content_sitewide_total_click_to_total_cart_std_3d_ratio",
    "content_sitewide_total_click_to_total_fav_std_3d_ratio",
    "content_sitewide_total_click_to_total_order_sum_3d_ratio",
    "content_sitewide_total_cart_to_total_order_sum_3d_ratio",
    "content_sitewide_total_fav_to_total_order_sum_3d_ratio",
    "content_sitewide_total_click_to_total_cart_sum_3d_ratio",
    "content_sitewide_total_click_to_total_fav_sum_3d_ratio",
    "content_sitewide_total_click_to_total_order_mean_7d_ratio",
    "content_sitewide_total_cart_to_total_order_mean_7d_ratio",
    "content_sitewide_total_fav_to_total_order_mean_7d_ratio",
    "content_sitewide_total_click_to_total_cart_mean_7d_ratio",
    "content_sitewide_total_click_to_total_fav_mean_7d_ratio",
    "content_sitewide_total_click_to_total_order_std_7d_ratio",
    "content_sitewide_total_cart_to_total_order_std_7d_ratio",
    "content_sitewide_total_fav_to_total_order_std_7d_ratio",
    "content_sitewide_total_click_to_total_cart_std_7d_ratio",
    "content_sitewide_total_click_to_total_fav_std_7d_ratio",
    "content_sitewide_total_click_to_total_order_sum_7d_ratio",
    "content_sitewide_total_cart_to_total_order_sum_7d_ratio",
    "content_sitewide_total_fav_to_total_order_sum_7d_ratio",
    "content_sitewide_total_click_to_total_cart_sum_7d_ratio",
    "content_sitewide_total_click_lag1",
    "content_sitewide_total_order_lag1",
    "content_sitewide_total_cart_lag1",
    "content_sitewide_total_fav_lag1",
    "content_search_rolling_mean_total_search_impression_7d",
    "content_search_rolling_std_total_search_impression_7d",
    "content_search_rolling_min_total_search_impression_7d",
    "content_search_rolling_max_total_search_impression_7d",
    "content_search_rolling_sum_total_search_impression_7d",
    "content_search_rolling_mean_total_search_click_7d",
    "content_search_rolling_std_total_search_click_7d",
    "content_search_rolling_min_total_search_click_7d",
    "content_search_rolling_max_total_search_click_7d",
    "content_search_rolling_mean_total_search_impression_3d",
    "content_search_rolling_std_total_search_impression_3d",
    "content_search_rolling_min_total_search_impression_3d",
    "content_search_rolling_max_total_search_impression_3d",
    "content_search_rolling_mean_total_search_click_3d",
    "content_search_rolling_std_total_search_click_3d",
    "content_search_rolling_min_total_search_click_3d",
    "content_search_total_search_impression_to_total_search_click_mean_3d_ratio",
    "content_search_total_search_impression_to_total_search_click_std_3d_ratio",
    "content_search_total_search_impression_to_total_search_click_sum_3d_ratio",
    "content_search_total_search_impression_to_total_search_click_mean_7d_ratio",
    "content_search_total_search_impression_to_total_search_click_std_7d_ratio",
    "content_search_total_search_impression_to_total_search_click_sum_7d_ratio",
    "content_search_total_search_impression_lag1",
    "content_search_total_search_click_lag1",
    "session_candidate_count",
    "rank_session_id_content_top_terms_total_search_impression_to_total_search_click_avg_ratio",
    "rank_session_id_content_top_terms_total_search_impression",
    "rank_session_id_content_top_terms_total_search_click",
    "rank_session_id_content_sitewide_total_click_to_total_cart_avg_ratio",
    "rank_session_id_content_sitewide_total_click",
    "rank_session_id_content_sitewide_total_cart",
    "rank_session_id_content_sitewide_total_click_to_total_order_avg_ratio",
    "rank_session_id_content_sitewide_total_order",
    "rank_session_id_content_sitewide_total_click_to_total_fav_avg_ratio",
    "rank_session_id_content_sitewide_total_fav",
    "rank_session_id_content_sitewide_total_cart_to_total_order_avg_ratio",
    "rank_session_id_content_search_total_search_impression_to_total_search_click_avg_ratio",
    "rank_session_id_content_search_total_search_impression",
    "rank_session_id_content_search_total_search_click",
    "content_top_terms_weighted_score",
    "rank_session_id_content_top_terms_weighted_score",
    "rank_session_id_content_sitewide_weighted_score",
    "content_search_weighted_score",
    "rank_session_id_content_search_weighted_score",
    "avg_content_search_and_sitewide_rank",
    "median_content_search_and_sitewide_rank",
    "total_content_search_and_sitewide_weighted_score",
    "rank_session_id_original_price_log",
    "rank_session_id_selling_price_log",
    "rank_session_id_discounted_price_log",
    "rank_session_id_discount_rate",
    "rank_session_id_selling_rate",
    "rank_session_id_content_rate_avg_bayesian",
    "rank_session_id_content_review_count_norm",
    "rank_session_id_content_review_wth_media_count_norm",
    "rank_session_id_wilson_score_rate_to_review",
    "rank_session_id_wilson_score_review_to_media",
]

cat_cols = [
    "user_gender",
    "leaf_category_name",
    "level1_category_name",
    "level2_category_name",
]

params_cat_ranker = {
    "iterations": 1000,
    "learning_rate": 0.05,
    "depth": 6,
    "loss_function": "YetiRank",
    "cat_features": cat_cols,
    "random_seed": 42,
    "verbose": 100,
}

weights = {
    "ordered": 9.0,
    "clicked": 0.5,
    "added_to_cart": 8.0,
    "added_to_fav": 1.8
}

print(f"Features (len={len(features)}) that will be trained with: {features}")

#### Training and Prediction

In [ ]:
train = train.sort(["session_id","content_id_hashed"])
test = test.sort(["session_id","content_id_hashed"])

### Negative Sampling
negative_limit = 1000
train = train.with_columns((pl.col("ordered") + pl.col("clicked") + pl.col("added_to_cart") + pl.col("added_to_fav")).alias("new_target"))
session_pivot = train.group_by(("session_id", "new_target")).agg(pl.count()).pivot(index="session_id", on="new_target", values="count")
negative_sessions = session_pivot.filter(pl.col("0") > negative_limit).select(pl.col("session_id"))
train_pd = train.to_pandas()
zero_sessions_limited = train_pd.loc[(train_pd["session_id"].isin(negative_sessions["session_id"])) & (train_pd["new_target"] == 0), :].groupby("session_id").sample(n=negative_limit).index
other_sessions = train_pd.loc[(train_pd["session_id"].isin(negative_sessions["session_id"])==False) | ((train_pd["session_id"].isin(negative_sessions["session_id"])) & (train_pd["new_target"] > 0)), :].index
selected_sessions = np.concatenate([zero_sessions_limited, other_sessions])
train = pl.from_pandas(train_pd.loc[selected_sessions, :].reset_index(drop=True)).sort(["session_id","content_id_hashed"])

train_sessions = train.select("session_id").to_numpy().flatten()

## Preprocess
train_X = train.select(features).to_pandas()
test_X = test.select(features).to_pandas()

train_y = train.select(
    (pl.col("ordered") * weights["ordered"]) + (pl.col("clicked") * weights["clicked"]) + (pl.col("added_to_cart") * weights["added_to_cart"]) + (pl.col("added_to_fav") * weights["added_to_fav"])
).to_numpy().flatten()

train_X[cat_cols] = train_X[cat_cols].fillna("unknown")
test_X[cat_cols] = test_X[cat_cols].fillna("unknown")

model_all_catbranker = CatBoostRanker(**params_cat_ranker)
model_all_catbranker.fit(train_X, train_y, group_id=train_sessions)

### Test Prediction
test = test.with_columns(
    pl.Series(f"prediction", model_all_catbranker.predict(test_X))
)

can_predictions = test.select(["session_id","user_id_hashed","content_id_hashed","prediction"])
can_predictions.write_csv(f"can_predictions.csv")

## Ensembling Solution

In [ ]:
ensemble_preds = can_predictions.join(pl.from_pandas(kaan_predictions), on=["session_id","user_id_hashed","content_id_hashed"], how="inner")
ensemble_preds = ensemble_preds.with_columns(
    (pl.col("prediction")*0.5 + pl.col("kaan_prediction")*0.5).alias("prediction")
)

ensemble_preds = ensemble_preds.sort(["session_id", "prediction"], descending=True)
ensemble_preds_submission = ensemble_preds.group_by("session_id").agg(
    pl.col("content_id_hashed").alias("prediction")
).with_columns(
    pl.col("prediction").list.join(" ")
)

ensemble_preds_submission.write_csv(f"final_submission_SKY.csv")